In [1]:
import json

In [2]:
from pyspark.sql import SparkSession

In [3]:
config = json.load(open("config.json"))

In [4]:
spark = SparkSession.builder\
    .config("spark.jars", config['JDBC_PATH']) \
    .config("spark.executor.extraClassPath", config['JDBC_PATH'])\
    .getOrCreate()

24/01/11 22:43:19 WARN Utils: Your hostname, przemyslaw-Latitude-E7450 resolves to a loopback address: 127.0.1.1; using 192.168.1.110 instead (on interface wlp2s0)
24/01/11 22:43:19 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
24/01/11 22:43:20 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/01/11 22:43:22 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [5]:
dbtable = "store"

In [6]:
df_store = spark.read.format("jdbc")\
        .option("url",config['SRC_DB_URL'])\
        .option("driver", "com.mysql.jdbc.Driver")\
        .option("dbtable", dbtable)\
        .option("user", config['SRC_DB_USER'])\
        .option("password", config['SRC_DB_PASSWORD'])\
        .load()

Loading class `com.mysql.jdbc.Driver'. This is deprecated. The new driver class is `com.mysql.cj.jdbc.Driver'. The driver is automatically registered via the SPI and manual loading of the driver class is generally unnecessary.


In [7]:
df_store.createOrReplaceTempView(dbtable)

In [8]:
dbtable = "staff"

In [9]:
df_staff = spark.read.format("jdbc")\
        .option("url",config['SRC_DB_URL'])\
        .option("driver", "com.mysql.jdbc.Driver")\
        .option("dbtable", dbtable)\
        .option("user", config['SRC_DB_USER'])\
        .option("password", config['SRC_DB_PASSWORD'])\
        .load()

In [10]:
df_staff.createOrReplaceTempView(dbtable)

In [11]:
dbtable = "address"

In [12]:
df_address = spark.read.format("jdbc")\
        .option("url",config['SRC_DB_URL'])\
        .option("driver", "com.mysql.jdbc.Driver")\
        .option("dbtable", dbtable)\
        .option("user", config['SRC_DB_USER'])\
        .option("password", config['SRC_DB_PASSWORD'])\
        .load()

In [13]:
df_address.createOrReplaceTempView(dbtable)

In [14]:
dbtable = "city"

In [15]:
df_city = spark.read.format("jdbc")\
        .option("url",config['SRC_DB_URL'])\
        .option("driver", "com.mysql.jdbc.Driver")\
        .option("dbtable", dbtable)\
        .option("user", config['SRC_DB_USER'])\
        .option("password", config['SRC_DB_PASSWORD'])\
        .load()

In [16]:
df_city.createOrReplaceTempView(dbtable)

In [17]:
dbtable = "country"

In [18]:
df_country = spark.read.format("jdbc")\
        .option("url",config['SRC_DB_URL'])\
        .option("driver", "com.mysql.jdbc.Driver")\
        .option("dbtable", dbtable)\
        .option("user", config['SRC_DB_USER'])\
        .option("password", config['SRC_DB_PASSWORD'])\
        .load()

In [19]:
df_country.createOrReplaceTempView(dbtable)

In [20]:
dim_store = spark.sql("""
SELECT
ROW_NUMBER() OVER(ORDER BY s.store_id) AS store_id,
s.store_id AS store_key,
s.store_manager_id AS store_manager_id,
CONCAT(stf.first_name, " ", stf.last_name) AS store_manager_name,
a.address_id AS store_address_id,
a.postal_code AS store_postal_code,
a.address AS store_address,
ci.city AS store_city,
co.country AS store_country,
True as store_is_active,
CURRENT_DATE() as store_valid_from,
NULL as store_valid_to
FROM store AS s
JOIN staff AS stf ON s.store_manager_id = stf.staff_id
JOIN address AS a ON s.address_id = a.address_id
JOIN city AS ci ON a.city_id = ci.city_id
JOIN country AS co ON ci.country_id = co.country_id
""")

In [21]:
dim_store.show()

24/01/11 22:43:39 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/01/11 22:43:39 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/01/11 22:43:39 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/01/11 22:43:43 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/01/11 22:43:43 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/01/11 22:43:43 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/01/11 2

+--------+---------+----------------+--------------------+----------------+-----------------+--------------------+------------+--------------+---------------+----------------+--------------+
|store_id|store_key|store_manager_id|  store_manager_name|store_address_id|store_postal_code|       store_address|  store_city| store_country|store_is_active|store_valid_from|store_valid_to|
+--------+---------+----------------+--------------------+----------------+-----------------+--------------------+------------+--------------+---------------+----------------+--------------+
|       1|        1|               2|       Viktor Soukup|               1|           00-001|   Żwirki i Wigury 1|    Warszawa|        Poland|           true|      2024-01-11|          NULL|
|       2|        2|              13|    Charles Oderwald|               6|           161 00|         Aviatická 6|      Prague|Czech Republic|           true|      2024-01-11|          NULL|
|       3|        3|               7|        

In [22]:
dim_store = dim_store.withColumn("store_valid_to", dim_store["store_valid_to"].cast("date"))

In [23]:
dim_store.printSchema()

root
 |-- store_id: integer (nullable = false)
 |-- store_key: long (nullable = true)
 |-- store_manager_id: long (nullable = true)
 |-- store_manager_name: string (nullable = true)
 |-- store_address_id: long (nullable = true)
 |-- store_postal_code: string (nullable = true)
 |-- store_address: string (nullable = true)
 |-- store_city: string (nullable = true)
 |-- store_country: string (nullable = true)
 |-- store_is_active: boolean (nullable = false)
 |-- store_valid_from: date (nullable = false)
 |-- store_valid_to: date (nullable = true)



In [24]:
dim_store.write.format("jdbc")\
        .option("url",config['DST_DB_URL'])\
        .option("driver", "com.mysql.jdbc.Driver")\
        .option("dbtable", "dim_store")\
        .option("user", config['DST_DB_USER'])\
        .option("password", config['DST_DB_PASSWORD'])\
        .save()

24/01/11 22:44:01 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/01/11 22:44:01 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/01/11 22:44:01 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/01/11 22:44:01 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/01/11 22:44:01 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/01/11 22:44:01 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/01/11 2